In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np
from keras.models import Model, Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.layers import *
import joblib

ProgressBar().register()

GRANULARITY = 0 # edit this. 1 -> 8x_10y_7z, .5 -> 16x_20y_14z, 0 -> exact
FRAMES_AHEAD = 15

In [ ]:
## NN shouldn't care about game time in OT
def fix_ot_secs(df):
    if len(df[df['seconds_remaining'] == -1.0]) > 0:
        print('already fixed')
        return df
    i=0
    try:
        while df.at[i, 'seconds_remaining'] >= df.at[i+1, 'seconds_remaining']:
            i += 1
        i += 1
        for j in range(i, len(df)):
            df.at[j, 'seconds_remaining'] = -1
        return df
    except:
        return df

def manip(df):
    df = df.drop(columns=['Unnamed: 0'])
    df = fix_ot_secs(df)
    df['0_pos_x_nf'] = df['0_pos_x'].shift(-1*FRAMES_AHEAD)
    df['0_pos_y_nf'] = df['0_pos_y'].shift(-1*FRAMES_AHEAD)
    df['0_pos_z_nf'] = df['0_pos_z'].shift(-1*FRAMES_AHEAD)
    df = df.head(len(df.index)-FRAMES_AHEAD)
    return df

In [ ]:
## combining replays ##
if GRANULARITY == 1:
    rootdir = '/home/zach/Files/Nas/ReplayModels/ReplayDataProcessing/RANKED_STANDARD/Replays/1400-1600/CSVs_8x_10y_7z/dask_CSVs'
elif GRANULARITY == .5:
    rootdir = '/home/zach/Files/Nas/ReplayModels/ReplayDataProcessing/RANKED_STANDARD/Replays/1400-1600/CSVs_16x_20y_14z/dask_CSVs'
elif GRANULARITY == 0:
    rootdir = '/home/zach/Files/Nas/ReplayModels/ReplayDataProcessing/RANKED_STANDARD/Replays/1400-1600/CSVs/dask_CSVs'

In [ ]:
all_dfs = dd.read_csv(rootdir+"/*.csv")

In [ ]:
all_dfs = all_dfs.map_partitions(manip)

In [ ]:
all_dfs = all_dfs.drop(columns=['Unnamed: 0'], errors="ignore")
rem_cols = []
for col in all_dfs.columns:
    if 'steer' in col or 'handbrake' in col \
    or 'active' in col or 'collect' in col \
    or 'ball_cam' in col or 'throttle' in col:
        rem_cols.append(col)
rem_cols.append("filepath")
rem_cols.append("filename")
all_dfs = all_dfs.drop(columns=rem_cols, errors="ignore")
all_dfs = all_dfs.dropna()
all_dfs = all_dfs.reset_index(drop=True)

In [ ]:
for col in all_dfs.columns:
    all_dfs[col] = (all_dfs[col] - all_dfs[col].mean()) / all_dfs[col].std()

In [ ]:
input_cols = [x for x in all_dfs.columns if x != '0_pos_x' \
              and x != '0_pos_y' and x != '0_pos_z']

In [ ]:
#x_train = all_dfs.filter(items=input_cols)
x_train = all_dfs.drop(columns=['0_pos_x', '0_pos_y', '0_pos_z'], errors="ignore")

In [ ]:
#y_train = all_dfs.filter(items=['0_pos_x', '0_pos_y', '0_pos_z'])
y_train = all_dfs.drop(columns=input_cols, errors="ignore")

In [ ]:
import dask.array as da

In [ ]:
x_train = da.from_array(np.asarray(x_train)).compute()

In [ ]:
y_train = da.from_array(np.asarray(y_train)).compute()

In [ ]:
def create_model(layers=[45], activation='sigmoid', optimizer='rmsprop'):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes, input_dim=x_train.shape[1]))
        else:
            model.add(Dense(nodes))
        model.add(Activation(activation))
    model.add(Dense(3))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [ ]:
print("CONSTRUCTING...")
model = KerasRegressor(build_fn=create_model, verbose=2)

layers = [[46], [68, 23]]
activations = ['relu']
batch_size = [128]
epochs = [90]
optimizers = ['rmsprop']

In [ ]:
param_grid = dict(layers=layers, activation=activations, batch_size=batch_size,
                  epochs=epochs, optimizer=optimizers)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    scoring='neg_mean_squared_error',
                    verbose=5, cv=5)

In [ ]:
print("FITTING...")
grid_result = grid.fit(x_train, y_train)

In [ ]:
print("SAVING...")
joblib.dump(grid_result, '/home/zach/Files/Nas/gd_obj_'+str(x_train.shape[0])+'_frames.pkl')

print(grid_result.best_score_)
print()
print(grid_result.best_params_)